# Bank marketing

**1. Project description**
+ **1.1 Goal**
+ **1.2 Data**
+ **1.3 Data provider**
+ **1.4 Software**

**2. Exploratory Data Analysis** 
+ **2.1 Loading the dataset**
+ **2.2 Missing values**
+ **2.3 Data types**
+ **2.4 Feature importance and analysis of target variable.**
+ **2.5 Doing the train/validation/test split**
+ **2.6 Encoding the categorical variables**
+ **2.7 Scaling**

**3. Model training** 
+ **3.1 Metric selection**
+ **3.2 Logistic Regression**
+ **3.3 Decision Tree**
+ **3.4 Random Forest**
+ **3.5 XGBoost**
+ **3.6 Model selection**

**4. Exporting the notebook to a python script**

## 1. Project description

### 1.1 Goal

The classification goal is to predict if the bank client will subscribe (yes/no) a fix term deposit (target variable 'deposit') after a marketing campaign. It might help the bank in designing promotions, new campaigns, and understanding controversial campaign results.

### 1.2 Data

###### Bank client data:
- age (numeric)
- job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
- marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
- education (categorical: 'primary', 'secondary', 'terciary','unknown')
- default: has credit in default? (categorical: 'no','yes','unknown')
- balance: not in the original data set, probably how much money the client has in the bank
- housing: has housing loan? (categorical: 'no','yes','unknown')
- loan: has personal loan? (categorical: 'no','yes','unknown')
- contact: contact communication type (categorical: 'cellular','telephone','unknown)
- month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
- day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
- duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

###### Campaign-related attributes:
- campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; '-1' means client was not previously contacted)
- previous: number of contacts performed before this campaign and for this client (numeric)
- poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

###### Output variable (desired target):
- deposit - has the client subscribed a term deposit? (binary: 'yes','no')

### 1.3 Data provider

Original dataset: [UCI Machine learning repository](https://archive.ics.uci.edu/ml/datasets/Bank%2BMarketing)

Here we use a subset of the dataset dowloaded from [Kaggle](https://www.kaggle.com/janiobachmann/bank-marketing-dataset). There are some minor differences:
+ this Kaggle dataset is balanced (in the Discussion tab at the Kaggle webpage in the link it is said that the resampling was probably done by bootstrapping). 
+ the variable 'education' has been simplified in the Kaggle dataset to 'primary', 'secondary', 'terciary', and 'unknown', the original values were: 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown'.
+ in the Kaggle dataset the variable 'pdays' takes the value '-1' (instead of '999' as in the original dataset) when the client was not previously contacted.
+ the Kaggle dataset has a column named 'balance' and does not have the 'Social and economic context attributes' (like euribor 3 moth rate) shown in the original dataset.

Citation: _A Data-Driven Approach to Predict the Success of Bank Telemarketing._ S. Moro, P. Cortez and P. Rita. Decision Support Systems, Elsevier, 62:22-31, June 2014

### 1.4 Software

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec

from IPython.display import display

%matplotlib inline

## 2. Exploratory Data Analysis    

### 2.1 Loading the dataset

In [ ]:
path = "/home/mmc/Desktop/DataTalks/mid_term_project/"  
df = pd.read_csv(path + "bank.csv")
df

### 2.2 Missing values

In [ ]:
max(df.isnull().sum())

There are not missing values in the dataset but many values are 'unknown'. We checked that the model performance is fine by having 'unknown' as a value. The model decision power might be reduced (i.e., predictions involving variables with 'unknown' values might not be very informative, see the Decision Tree Classifier section), however we prefer to do not replace these values by NaN or zeros.

In [ ]:
df[df == 'unknown'].count()

### 2.3 Data types

In [ ]:
df.dtypes

Some categorical variables can easily be converted to numeric for the EDA:

In [ ]:
# Replace with numbers 0 (negative or 'no') and 1 (positive or 'yes')
df.default = (df.default == 'yes').astype(int)
df.loan = (df.loan == 'yes').astype(int)
df.housing = (df.housing == 'yes').astype(int)
df.deposit = (df.deposit == 'yes').astype(int)

In [ ]:
categorical = df.select_dtypes(include=['object']).columns.tolist()
numerical = df.select_dtypes(include=['int64']).columns.tolist()

### 2.4 Feature importance

To identify which features affect the target variable 'deposit'.

##### Numerical variables

##### A. Range of values and basic stats

In [ ]:
df.describe().round(2)

Different columns show very different range of values, we will need to scale (see below, after the data split).

##### B. Pair plots

In [ ]:
# This visualization takes a while, set the if as "True" to run it
if True:#False: 
    colors = ["#f58e00", "g"]
    labels = "No", "Yes"
    hue_order = [0, 1]

    ax = sns.pairplot(df, hue='deposit', palette=colors, hue_order=hue_order)
    ax.fig.suptitle('Pair Plots to show potential dependencies between numerical features (double-click to enlarge)',
                family='Serif', size=30, ha='center', weight='bold', y = 1.01)
    plt.show()

The pair plots do not show obvious dependencies.

##### C. Frequency distribution

The diagonal of the pair plots above show the frequency distribution of the numerical features. The target variable 'deposit' is quite balanced:

In [ ]:
plt.style.use('seaborn-whitegrid')
df.deposit.hist(bins=20, figsize=(2,4), color="#f58e00")
plt.suptitle('Distribution of the target variable', family = 'Serif', 
             size = 15,weight = 'bold')
plt.show()

In [ ]:
df.deposit.value_counts(normalize=True) 

##### D. Correlation by the [Pearson coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)

In [ ]:
matrix_corr = df.corr().round(3)

In [ ]:
plt.figure(figsize=(25,10))
plt.rc('font', size=15) 
sns.heatmap(matrix_corr,annot=True,linewidths=.5, cmap="Greens")
plt.title('Heatmap showing correlations between numerical data')
plt.show() 

The feature 'duration', i.e., the duration of the last contact, shows the higher correlation with the target deposit. However, as mentioned in the data provider section above, it might be because if 'duration'=0 then 'deposit'='no', and the call duration is not known before a call is performed. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model (see the section Model Selection of a comparison of the results including and not including this feature).

Apart from 'duration', the next highest correlations in the numerical features are a positve correlation with 'pdays' (days from last contact) and 'previous' (contacts before this campaign) and negative correlations with 'housing', 'campaign' (contacts in this campaign) and 'loan' (people with loans might not engage to the fix term deposit).

##### E. ROC AUC for feature importance

The [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) is the probability that a randomly selected positive example (someone that will agree to open a fix term deposit) has a higher score (higher probability to be predicted as positive or higher feature value if, like here, we want to use ROC AUC for feature important), than a randomly selected negative example (someone that will not open a fix term deposit).

In [ ]:
feature_scores = []

for col in numerical:       
    auc = roc_auc_score(df.deposit, df[col]) 
    if auc < 0.5: # in case the feature is negatively correlated with the target
        auc = roc_auc_score(df.deposit, -df[col]) 
    feature_scores.append((col, auc))

columns = ['feature', 'ROC_AUC']
df_scores = pd.DataFrame(feature_scores, columns=columns)
df_scores.sort_values(by=['ROC_AUC'],ascending=False).reset_index(drop = True)

Again, apart from 'duration', that can be problematic, as the data provider mentioned above, the highest ROC AUC in the numerical features are 'housing', 'previous' (contacts before this campaign), and 'pdays' (days from last contact). Now also 'balance' shows relevance, even higher than 'loan' (people with loans might not engage to the fix term deposit) and 'campaign' (number of contacts in this campaign).

##### Categorical variables

##### A. Mean difference and risk ratio

In [ ]:
for c in categorical:
    print(c)
    df_group = df.groupby(c).deposit.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - df.deposit.mean() # mean difference
    df_group['risk'] = df_group['mean'] / df.deposit.mean() # risk ratio
    display(df_group)
    print()

It seems that 'month' (specially 'December' and 'March' with a risk ratio about 90% above average), 'job' (specially 'retired' and 'student') and previous outcome 'poutcome' (specially 'success') significatively affect if there is a deposit or not. The 'unknown' value of the feature 'contact' also shows an effect on the target variable, although this might not be very informative (low decision power).

##### B. Mutual Information

From [information theory](https://en.wikipedia.org/wiki/Mutual_information), for categorical variables, it tells us how much we can learn about one variable if we know the value of another.

In [ ]:
def mut_inf_deposit_score(series):
    return mutual_info_score(series, df.deposit)

# apply the function column-wise
MutInf = df[categorical].apply(mut_inf_deposit_score)
MutInf.sort_values(ascending=False) # to sort it

##### Feature importance summary

The next highest correlations in the numerical features are a positive correlation with **'pdays'** (days from last contact) and **'previous'** (contacts before this campaign) and negative correlations with **'housing'** and **'loan'** (people with loans might not engage to the fix term deposit) and **'campaign'** (number of contacts in this campaign).

The previous outcome **'poutcome'**, **'contact'**, and **'job'**, seem to be the most relevant categorical features. 

We still use all of them (see the Model Selection section about the results if we dismiss 'duration', as suggested by the data provider) since 15 is a suitable number of features. So far, the 'unknown' values has not being converted to NaNs.

In [ ]:
df_select = df.copy() 
#del df_select['duration']  
categorical = df_select.select_dtypes(include=['object']).columns.tolist()
numerical = df_select.select_dtypes(include=['int64']).columns.tolist()
numerical.remove('deposit')

Uncomment the removal of the feature 'duration' to check how is performance without this feature, which is defined as problematic by the dat aprovider, see above. When uncommented, the final XGBoost ROC AUC in the Model selection section is ~0.787.

### 2.5 Doing the train/val/test split

In [ ]:
# separate train + validation (= full) and test
df_full_train, df_test = train_test_split(df_select, test_size=0.2, random_state=1)
# now split the full into train and val, it should be the 20% of the 80%, which is 20/80=1/4=0.25
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1) 
len(df_train), len(df_val), len(df_test)

In [ ]:
# reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

# separate the target
y_train = df_train.deposit.values
y_val = df_val.deposit.values

# remove the target from the features
del df_train['deposit']
del df_val['deposit']


### 2.6 Encoding the categorical variables

We use Scikit-Learn DictVectorizer to encode categorical features (it takes a dictionary and convert it to a vector (numpy.array)). It is One-hot encoding (OHE) method to converts the categorical features in binary (it would not affect the numerical ones), in as much columns as values the categorical variable takes.

In [ ]:
dv = DictVectorizer(sparse=False) # False bcs is not a sparse matrix (we do not have many zeros)

# TRAIN
train_dict = df_train[categorical].to_dict(orient='records') # records = to do it row-wise, not col-wise
X_train_cat = dv.fit_transform(train_dict) # make it a vector

# VAL
val_dict = df_val[categorical].to_dict(orient='records')
X_val_cat = dv.transform(val_dict)

### 2.7 Scaling the numerical variables

We use Scikit-Learn StandardScaler to scale the numerical features (otherwise columns with values in a higher range would have more representation and the model does not converge).

In [ ]:
scaler = StandardScaler()

# TRAIN
X_train_num = df_train[numerical].values
X_train_num = scaler.fit_transform(X_train_num)

# VAL
X_val_num = df_val[numerical].values
X_val_num = scaler.transform(X_val_num)

We join the numerical and categorical matrices:

In [ ]:
# TRAIN
X_train = np.column_stack([X_train_num, X_train_cat])

# VAL
X_val = np.column_stack([X_val_num, X_val_cat])

## 3 Model training

### 3.1 Metric selection

As the target in the dataset is quite balanced, the performance metrics we will use are:
+ [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification) i.e., the proportion of right predictions, and 
+ [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic), as mentioned above, the probability that a randomly selected positive example (someone that will actually open a fix term deposit) has a higher score (higher probability to be predicted as positive or higher feature value if we want to use ROC AUC for feature important) than a randomly selected negative example (someone that will not open a fix term deposit).

### 3.2 Logistic Regression

A linear regression as we studied in school that applies a 'sigmoid' filter to the predictions thus the results are binary values.

In [ ]:
LR = LogisticRegression(C=1.0, random_state=42) 
# solver='lbfgs' is the default solver in newer version of sklearn
# the smallest the C, the stronger the regularizations (oposite to alpha o r)

LR.fit(X_train, y_train)
y_pred = LR.predict_proba(X_val)[:, 1] # left col is neg deposit (0), right col is pos deposit (1)
  
thresholds = np.linspace(0, 1, 5)
LR_scores = []
print('thres', 'acc')

for t in thresholds: # above threshold the probability becomes 1, below, is zero
    acc = accuracy_score(y_val, y_pred >= t) # it compares the 0/1 in y_val with the False/True of y_pred>=t
    print('%.2f %.3f' % (t, acc))
    LR_scores.append(acc)

Best threshold is t = 0.5. A dummy threshold, t=0, means that the model predicts that none of the clients make a fix term deposit, and it is correct 50% of the times since the data are quite balanced.

In [ ]:
t = 0.5

# TRAIN
y_pred = LR.predict_proba(X_train)[:, 1]
acc = accuracy_score(y_train, y_pred >= t) 
auc = roc_auc_score(y_train, y_pred)
print('For the training dataset:','ACC:', acc.round(3), 'ROC AUC:', auc.round(3))

# VAL
y_pred = LR.predict_proba(X_val)[:, 1]
LR_acc = accuracy_score(y_val, y_pred >= t) 
LR_auc = roc_auc_score(y_val, y_pred)
print('For the validation dataset:','ACC:', LR_acc.round(3), 'ROC AUC:', LR_auc.round(3))

The performances of the model on the training and validation datasets are similar, thus there is not so much overfitting. The scores are good. Instead of tuning the parameters of the logistic regresion, let us try to improve the performance with non linear models.

### 3.3 Decision Tree Classifier

Decision trees learn if-then-else rules from data where finding the best split (true or false in the condition) means to select the least impure split. This algorithm can overfit, that's why we control it by limiting the max depth and the size of the group. Let us find the best threshold to calculate the accuracy (as we did for the Logistic Regression model), then we tune the tree parameters.

In [ ]:
DT = DecisionTreeClassifier(max_depth = 6, random_state=1)
DT.fit(X_train, y_train)

y_pred = DT.predict_proba(X_val)[:, 1] # left col is neg deposit (0), right col is pos deposit (1)
  
thresholds = np.linspace(0, 1, 5)
DT_scores =[]
print('thres', 'acc')

for t in thresholds: # above threshold the probability becomes 1, below, is zero
    score = accuracy_score(y_val, y_pred >= t) # it compares the 0/1 in y_val with the False/True of y_pred>=t
    print('%.2f %.3f' % (t, score))
    DT_scores.append(score)
    
print('the ROC AUC is', roc_auc_score(y_val, y_pred).round(3))

Threshold t=0.5 seems to be the optimal choice again, thus we will just use it from now on. Let us find the best parameters for the decison tree classifier.

In [ ]:
DT_scores = []
t = 0.5

for depth in [4, 5, 6]:
    for s in [1, 5, 10, 15, 20, 500, 100, 200]:
        DT = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=s, random_state = 1)
        DT.fit(X_train, y_train)

        y_pred = DT.predict_proba(X_val)[:, 1]
        acc = accuracy_score(y_val, y_pred >= t) 
        auc = roc_auc_score(y_val, y_pred)
        
        DT_scores.append((depth, s, acc, auc))
        
columns = ['max_depth', 'min_samples_leaf', 'acc','auc']
df_scores = pd.DataFrame(DT_scores, columns=columns)
df_scores_pivot = df_scores.pivot(index='min_samples_leaf', columns=['max_depth'], values=['acc','auc'])
sns.heatmap(df_scores_pivot, annot=True, fmt=".3f")

It seems that max_depth=6 and min_samples_leaf=5 are good enought (the performance on the training and the validation data should not be too apart or the model will overfit and not generalize well for the final test dataset).

In [ ]:
t = 0.5
DT = DecisionTreeClassifier(max_depth=6, min_samples_leaf=5, random_state =1)
DT.fit(X_train, y_train)

# TRAIN
y_pred = DT.predict_proba(X_train)[:, 1]
acc = accuracy_score(y_train, y_pred >= t) 
auc = roc_auc_score(y_train, y_pred)
print('For the training dataset:','ACC:', acc.round(3), 'ROC AUC:', auc.round(3))

# VAL
y_pred = DT.predict_proba(X_val)[:, 1]
DT_acc = accuracy_score(y_val, y_pred >= t) 
DT_auc = roc_auc_score(y_val, y_pred)
print('For the validation dataset:','ACC:', DT_acc.round(3), 'ROC AUC:', DT_auc.round(3))

The performances on the training and validation datasets are similar, there is not overfitting. 
Let us see how the leafs and branches look like:

In [ ]:
print(export_text(DT, feature_names=numerical + dv.get_feature_names()))

Apart from **'duration'** (which according to the data provider could be problematic, see the notebook intro), it seems that the features with more decision power in the tree are:
+ **'poutcome'**, i.e. if the previous campaign worked with this client or not, it also showed a high feature importance in the EDA section (although it can be not informative since there are many 'unknown' values),
+ **'contact'**, i.e., how was the cliented contacted (although it can be not informative since there are many 'unknown' values), and 
+ **'campaign'**, i.e., how many times the client was contacted during the campaign, it showed some (negative) correlation with the target in the EDA, and there are not 'unknown' values, it seems to be an important tree branch with high prediction power in combination with 'pdays'. 

Surprisingly, **'housing'** showed higher correlation with 'deposit' and higher ROC AUC than 'campaign', but here it seems to be as relevant as **'month'** (some 'month' values showed a very high risk ratio in the EDA).

### 3.4 Random Forest

Let us go for ensemble learning and run in parallel many trees and take the average result. Random forest is a way of combininig multiple decision trees. It should have a diverse set of models to make good predictions (although here we will not explore the 'bootstrap' option). First we find the best parameters: first the max_depth, then the min_samples_leaf, and finally the n_estimators.

In [ ]:
#%%timeit  # it takes about 1min
if True:#False:
    RF_scores = []
    t =0.5

    for d in [5, 10, 15]:
        for n in range(10, 201, 10):
            RF = RandomForestClassifier(n_estimators=n,
                                        max_depth=d,
                                        random_state=1)
            RF.fit(X_train, y_train)

            y_pred = RF.predict_proba(X_val)[:, 1]
            acc = accuracy_score(y_val, y_pred >= t) 
            auc = roc_auc_score(y_val, y_pred)

            RF_scores.append((d, n, acc, auc))

    columns = ['max_depth', 'n_estimators', 'acc','auc']
    df_scores = pd.DataFrame(RF_scores, columns=columns)
    df_scores

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
fig.tight_layout()

for d in [5, 10, 15]:
    df_subset = df_scores[df_scores.max_depth == d]
    
    axes[0].plot(df_subset.n_estimators, df_subset.acc,
             label='max_depth=%d' % d)
    axes[0].set_title('Accuracy')
    axes[0].set_xlabel('no. of trees')

    axes[1].plot(df_subset.n_estimators, df_subset.auc,
             label='max_depth=%d' % d)
    axes[1].set_title('ROC AUC')
    axes[1].set_xlabel('no. of trees')

plt.legend()


In [ ]:
# %%timeit it takes about 1min

if True:#False:
    max_depth = 15
    t=0.5

    RF_scores = []

    for s in [1, 3, 5, 10, 50]:
        for n in range(10, 201, 10):
            RF = RandomForestClassifier(n_estimators=n,
                                        max_depth=max_depth,
                                        min_samples_leaf=s,
                                        random_state=1)
            RF.fit(X_train, y_train)

            y_pred = RF.predict_proba(X_val)[:, 1]
            acc = accuracy_score(y_val, y_pred >= t)         
            auc = roc_auc_score(y_val, y_pred)

            RF_scores.append((s, n, acc, auc))

In [ ]:
columns = ['min_samples_leaf', 'n_estimators', 'acc','auc']
df_scores = pd.DataFrame(RF_scores, columns=columns)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
fig.tight_layout()

colors = ['black', 'blue', 'orange', 'red', 'grey']
values = [1, 3, 5, 10, 50]

for s, col in zip(values, colors):
    df_subset = df_scores[df_scores.min_samples_leaf == s]
    
    axes[0].plot(df_subset.n_estimators, df_subset.acc,
             color=col,
             label='min_samples_leaf=%d' % s)
    axes[0].set_title('Accuracy')
    axes[0].set_xlabel('no. of trees')

    axes[1].plot(df_subset.n_estimators, df_subset.auc,
             color=col,
             label='min_samples_leaf=%d' % s)
    axes[1].set_title('ROC AUC')
    axes[1].set_xlabel('no. of trees')

plt.legend()

The best parameters are:

In [ ]:
n_estimators = 150
min_samples_leaf = 1
max_depth = 15
t=0.5

RF = RandomForestClassifier(n_estimators=n_estimators,
                            max_depth=max_depth,
                            min_samples_leaf=min_samples_leaf,
                            random_state=1)
RF.fit(X_train, y_train)

# TRAIN
y_pred = RF.predict_proba(X_train)[:, 1]
acc = accuracy_score(y_train, y_pred >= t) 
auc = roc_auc_score(y_train, y_pred)
print('For the training dataset:','ACC:', acc.round(3), 'ROC AUC:', auc.round(3))

# VAL
y_pred = RF.predict_proba(X_val)[:, 1]
RF_acc = accuracy_score(y_val, y_pred >= t) 
RF_auc = roc_auc_score(y_val, y_pred)
print('For the validation dataset:','ACC:', RF_acc.round(3), 'ROC AUC:', RF_auc.round(3))

The performance on the training dataset is better than on the validation dataset, the model seems to overfit the target variable. Let us reduce the value of some parameters:

In [ ]:
n_estimators = 125 #150
min_samples_leaf = 10 #3
max_depth = 10 #15
t=0.5

RF = RandomForestClassifier(n_estimators=n_estimators,
                            max_depth=max_depth,
                            min_samples_leaf=min_samples_leaf,
                            random_state=1)
RF.fit(X_train, y_train)

# TRAIN
y_pred = RF.predict_proba(X_train)[:, 1]
acc = accuracy_score(y_train, y_pred >= t) 
auc = roc_auc_score(y_train, y_pred)
print('For the training dataset:','ACC:', acc.round(3), 'ROC AUC:', auc.round(3))

# VAL
y_pred = RF.predict_proba(X_val)[:, 1]
RF_acc = accuracy_score(y_val, y_pred >= t) 
RF_auc = roc_auc_score(y_val, y_pred)
print('For the validation dataset:','ACC:', RF_acc.round(3), 'ROC AUC:', RF_auc.round(3))

Now the performance on the validation is similar than with the optimal parameters set, but it is closer to the performance on the training dataset.

### 3.5 XGBoost

Gradient boosting trains model sequentially: each model tries to fix errors of the previous model. XGBoost is an implementation of gradient boosting. Then, we will run an ensemble of trees but not in parallel, thus the next tree can learn from the previous tree.

In [ ]:
features = numerical + dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 6, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=10)

# TRAIN
y_pred = model.predict(dtrain)
acc = accuracy_score(y_train, y_pred >= t) 
auc = roc_auc_score(y_train, y_pred)
print('For the training dataset:','ACC:', acc.round(3), 'ROC AUC:', auc.round(3))

# VAL
y_pred = model.predict(dval)
xgb_acc = accuracy_score(y_val, y_pred >= t) 
xgb_auc = roc_auc_score(y_val, y_pred)
print('For the validation dataset:','ACC:', xgb_acc.round(3), 'ROC AUC:', xgb_auc.round(3))

Let us define the parameter ranges to find the best parameter values, first 'eta', then 'max_depth' and then
'min_child_weight'.

In [ ]:
# performance monitoring: 
# after each round (i.e., after each new tree is trained) we inmediatly evaluate on the val dataset 
watchlist = [(dtrain, 'train'), (dval, 'val')]  

In [ ]:
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)
# from the results in each round we can identify overfitting

In [ ]:
xgb_params = {
    'eta': 0.1, #default is 0.3
    'max_depth': 6, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)

In [ ]:
xgb_params = {
    'eta': 0.01, #default is 0.3
    'max_depth': 6, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)

With learnin grate 'eta = 0.1' in the round [50] we get 'train-auc:0.85489' and 'val-auc:0.77817', meaning that there is not so much overfitting and it is a reasonable good score. Let us tune the 'max_depth':

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 4, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 3, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)

The best result with 'eta = 0.3' when varying 'max_depth' and avoiding overfitting seems to be 'max_depth=4' that in the round 25 gives 'train-auc: 0.9446' and 'val-auc: 0.9142'. Now we vary 'min_child_weight':

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 4, # default is 6
    'min_child_weight': 10, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 4, # default is 6
    'min_child_weight': 30, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
# train the XGB
model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                verbose_eval=5, # we print the evaluation every 5 trees/iterations
                evals=watchlist)
# apply to val
y_pred = model.predict(dval)

The default value 'min_child_weight = 1' gave the best results, in 25 rounds, both performances were high and similar: train-auc:0.94462 and val-auc:0.91423. Then, we select the following parameter set:

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 4, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=25)
y_pred = model.predict(dval)
xgb_auc =roc_auc_score(y_val, y_pred)
xgb_auc

We can analyze the feature importances very clearly by using the plot_importance() method. This gives the relative importance of all the features in the dataset.

In [ ]:
plt.figure()
plt.rcParams['figure.dpi'] = 120 
plt.rcParams.update({'font.size': 8})
xgb.plot_importance(model)
plt.show()

Interestingly, with the F-score as metric, the features 'age' and 'balance' become very relevant, which makes a lot of sense. Besides, we show in the previous analysis, that 'duration' is significatively important, as 'pdays' and a success in the previous campaign or 'poutcome=success'.

### 3.6 Model selection 

We will choose and train the final model.

In [ ]:
print('Model performance ROC AUC on the validation dataset:')
print()
print('XGBoost', xgb_auc.round(3))
print('Random Forest', RF_auc.round(3))
print('Logistic Regresssion:', LR_auc.round(3))
print('Decission Tree Classifier:', DT_auc.round(3))

When we removed the feature 'duration' following the data provider recommendation (see the data descripcion above), we obtained the following ROC AUC on the validation dataset (the code is not shown here, to reproduce the results, uncomment the line where we deleted the 'duration' feature just before splitting):
+ XGBoost 0.781 ('eta': 0.1, 'max_depth': 4, 'min_child_weight': 1, and 70 rounds)
+ Random Forest 0.779 (same parameters than here)
+ Logistic Regresssion: 0.749
+ Decission Tree Classifier: 0.725 (same parameters than here).

**The best performer is XGBboost in both cases.** We continue with the analysis that includes all the features.

We train it on the 'full_train' dataset and test it on the test dataset with a k-fold cross validation. Let us prepare the full_train and test datasets:

In [ ]:
# TRAIN
df_full_train = df_full_train.reset_index(drop=True) # reset index after splitting shuffling
y_full_train = df_full_train.deposit.values

#del df_full_train['deposit'] # remove target
    
full_train_dict = df_full_train[categorical].to_dict(orient='records')
X_full_train_cat = dv.fit_transform(full_train_dict) # encode the categorical features

X_full_train_num = df_full_train[numerical].values
X_full_train_num = scaler.fit_transform(X_full_train_num) # scale the numerical features

X_full_train = np.column_stack([X_full_train_num, X_full_train_cat]) # join the matrices

# TEST
df_test = df_test.reset_index(drop=True) # reset index after splitting shuffling
y_test = df_test.deposit.values

del df_test['deposit'] # remove target
    
test_dict = df_test[categorical].to_dict(orient='records')
X_test_cat = dv.transform(test_dict) # encode the categorical features

X_test_num = df_test[numerical].values
X_test_num = scaler.transform(X_test_num) # scale the numerical features

X_test = np.column_stack([X_test_num, X_test_cat]) # join the matrices

Create the DMatrices for XGBoost

In [ ]:
dfulltrain = xgb.DMatrix(X_full_train, label=y_full_train,
                    feature_names=numerical + dv.get_feature_names())

dtest = xgb.DMatrix(X_test, feature_names=numerical + dv.get_feature_names())

Train and apply the model.

In [ ]:
xgb_params = {
    'eta': 0.3, #default is 0.3
    'max_depth': 4, # default is 6
    'min_child_weight': 1, # default is 1
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc', # otherwise it uses logloss
    
    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dfulltrain, num_boost_round=25)
y_pred = model.predict(dtest)
xgb_auc =roc_auc_score(y_test, y_pred)

In [ ]:
xgb_auc

The performance on the test dataset is good and similar to the previous performances on the training and validation datasets. Let us see if it is robust with a k-fold cross-validation: we evaluate the same model on different subsets of data (k-subsets, called fold) and we get the average prediction and the spread of the predictions. It is a powerful preventative measure against overfitting thus the model might be capable of generalization.

In [ ]:
n_fold = 5
dfulltrain = xgb.DMatrix(X_full_train, label=y_full_train, feature_names=features)
cv_results = xgb.cv(dtrain=dfulltrain, params=xgb_params, nfold=n_fold, num_boost_round=25,as_pandas=True,seed =1)
cv_results.iloc[-1] # results of the last xgb round

The final model is quite stable. Let us save it.

## 4. Exporting to notebook to a python script

The logic for training the model is exported to a separate script:
+ in 'File', 'Download as', 'as Python (.py)' 

and then we save the trained model in the computer with pickle.

See the README.md of the [Github repo](https://github.com/MMdeCastro/ml-zoomcamp/tree/main/Midterm_project) for the next steps:
+ Model deployment with Flask
+ Dependency and enviroment management with pipenv*
+ Containerization with Docker
+ Cloud deployment